In [1]:
import pandas as pd
import numpy as np

In [2]:
# Before you doing this, you need to remove the first line(header) of the below file
match_best = pd.read_csv('Algorithm/match_best.csv', header=None)
match_second = pd.read_csv('Algorithm/match_second.csv', header=None)
match_third = pd.read_csv('Algorithm/match_third.csv', header=None)
raw_data = pd.read_csv('Algorithm/raw_data.csv', header=None)
Training_data = pd.read_csv('Algorithm/training_data.csv', header=None)
Testing_data = pd.read_csv('Algorithm/testing_data.csv', header=None)

In [3]:
def S(x) :
    if x > -1 and x < 1:
        return x
    elif x >= 1 :
        return 1
    else :
        return -1

In [4]:
X = Training_data.ix[0:, 0:113]
Y = Training_data.ix[0:, 114]
X_Mul_Y = X.mul(Y, axis=0)
Sum_X_Mul_Y = X_Mul_Y.sum()
X_T = X.transpose()
X_T_Dot_X = X_T.dot(X)
Parameter = X_T_Dot_X.as_matrix()

In [5]:
def Optimization(parameter_l, parameter_u, parameter_g) :
    
    U = match_second - match_third
    data = np.identity(115)
    
    I = pd.DataFrame(data=data[1:,1:])
    C = pd.concat([parameter_l * I, parameter_u * U],ignore_index=True)
    
    # initial w 
    random_num = np.random.uniform(-1, 1, size=114)
    w = pd.DataFrame(data=random_num)
    
    C_Dot_W = C.dot(w) / parameter_g
    
    A = C_Dot_W[0].apply(S)
    # C_T : C 's transpose
    C_T = C.transpose()
    C_T_Dot_A = 0.5 * C_T.dot(A)
    
    Target = Sum_X_Mul_Y - C_T_Dot_A
    Target = Target.as_matrix()
    
    w = np.linalg.solve(Parameter, Target)
    
    iterations = 0
    delta_A = 1000
    delta_w = 1000
    bound = 0.001

    while delta_A > bound and delta_w > bound :
        iterations += 1
        if iterations > 20 :
            return None
        
        C_Dot_W = C.dot(w) / parameter_g

        # update vector A
        A_now = C_Dot_W.apply(S)

        C_T_Dot_A = 0.5 * C_T.dot(A_now)
        Target = Sum_X_Mul_Y - C_T_Dot_A
        Target = Target.as_matrix()

        # update vector w
        w_now = np.linalg.solve(Parameter, Target)

        delta_A = np.linalg.norm(A_now-A)
        delta_w = np.linalg.norm(w_now-w)

        A = A_now
        w = w_now

        print 'iterations:' + str(iterations) + ' ' + \
                'delta_A:' + str(delta_A) + ' ' + \
                'delta_w:' + str(delta_w)  
    
    return w      

In [6]:
test_features = Testing_data.ix[0:, 0:113]
test_flag = Testing_data.ix[0:, 114]

In [7]:
def Evaluation(w) :
    
    result = np.dot(test_features, w)
    predict_flag = result > 0
    
    true_positive = 0.0
    true_negative = 0.0
    false_positive = 0.0
    false_negative = 0.0

    for i in range( len(predict_flag) ):
        if predict_flag[i] == True and test_flag[i] == 1 :
            true_positive += 1
        if predict_flag[i] == False and test_flag[i] == -1 :
            true_negative += 1
        if predict_flag[i] == True and test_flag[i] == -1 :
            false_positive += 1
        if predict_flag[i] == False and test_flag[i] == 1 :
            false_negative += 1
            
    ACC = (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative)
    PRE = true_positive / (true_positive + false_positive)
    REC = true_positive / (true_positive + false_negative)
    F1 = 2*( PRE * REC) / (PRE + REC)
    
    return F1 # ACC , PRE, REC

In [8]:
w = Optimization(0.01,1.32,0.04)
print w

iterations:1 delta_A:92.1843375231 delta_w:0.581102921085
iterations:2 delta_A:51.6433320108 delta_w:0.300130487158
iterations:3 delta_A:39.2463980443 delta_w:0.187078543693
iterations:4 delta_A:30.2199474513 delta_w:0.13043526635
iterations:5 delta_A:23.4744019334 delta_w:0.091931898693
iterations:6 delta_A:18.3126516311 delta_w:0.0670789742744
iterations:7 delta_A:14.3703197003 delta_w:0.04984833036
iterations:8 delta_A:11.1236179755 delta_w:0.0380186817406
iterations:9 delta_A:8.92079067695 delta_w:0.0301028871759
iterations:10 delta_A:7.2301157891 delta_w:0.0247927008497
iterations:11 delta_A:6.00047270427 delta_w:0.0208186402485
iterations:12 delta_A:5.04546135664 delta_w:0.0178293930653
iterations:13 delta_A:4.31890289274 delta_w:0.0153921713548
iterations:14 delta_A:3.73655748573 delta_w:0.0134572991551
iterations:15 delta_A:3.27998919969 delta_w:0.0118680906324
iterations:16 delta_A:2.89612307402 delta_w:0.0105419407613
iterations:17 delta_A:2.58751985567 delta_w:0.009470954666

In [9]:
# just for testing 
parameter_l = 0.01 
parameter_u = 0.0
parameter_g = 0.0
Performace = {}
loops = 0

while parameter_l < 100.0:

    if parameter_l < 0.1 :
        parameter_l += 0.03
    elif parameter_l >= 0.1 and parameter_l < 1.0:
        parameter_l += 0.3
    elif parameter_l >= 1.0 and parameter_l < 10.0:
        parameter_l += 3.0
    else :
        parameter_l += 30.0
    
    parameter_u = 0.0
    parameter_g = 0.0 

    while parameter_u < 100.0:

        if parameter_u < 0.1 :
            parameter_u += 0.03
        elif parameter_u >= 0.1 and parameter_u < 1.0:
            parameter_u += 0.3
        elif parameter_u >= 1.0 and parameter_u < 10.0:
            parameter_u += 3.0
        else :
            parameter_u += 30.0

        parameter_g = 0.0 

        while parameter_g < 100.0 :
            loops += 1
            print loops 

            if parameter_g < 0.1 :
                parameter_g += 0.03
            elif parameter_g >= 0.1 and parameter_g < 1.0 :
                parameter_g += 0.3
            elif parameter_g >= 1.0 and parameter_g < 10.0 :
                parameter_g += 3.0
            else :
                parameter_g += 30.0

            key = str(parameter_l) + ',' + str(parameter_u) + str(',') + str(parameter_g)
            print key
            w = Optimization(0.01, parameter_u, parameter_g)
            # print Evaluation(w)
            if w is not None :
                Performace[key] =  Evaluation(w) 


1
0.04,0.03,0.03
iterations:1 delta_A:49.0740366822 delta_w:0.00719280666113
iterations:2 delta_A:0.168931313142 delta_w:3.99966009694e-05
2
0.04,0.03,0.06
iterations:1 delta_A:38.4137479478 delta_w:0.00690545668133
iterations:2 delta_A:0.0771389152203 delta_w:1.78045670644e-05
3
0.04,0.03,0.09
iterations:1 delta_A:27.1382896848 delta_w:0.0044754952552
iterations:2 delta_A:0.0344221366327 delta_w:7.24455202983e-06
4
0.04,0.03,0.12
iterations:1 delta_A:24.324080374 delta_w:0.00427953259211
iterations:2 delta_A:0.0236151564348 delta_w:5.51541907953e-06
5
0.04,0.03,0.42
iterations:1 delta_A:6.71514829709 delta_w:0.00126184363667
iterations:2 delta_A:0.00220547348042 delta_w:5.1995069297e-07
6
0.04,0.03,0.72
iterations:1 delta_A:3.1179509807 delta_w:0.000574693341008
7
0.04,0.03,1.02
iterations:1 delta_A:3.1936021727 delta_w:0.000587849753885
8
0.04,0.03,4.02
iterations:1 delta_A:0.750356799378 delta_w:0.000116695825847
9
0.04,0.03,7.02
iterations:1 delta_A:0.383239412881 delta_w:6.2639456

In [10]:
import operator
sorted_Performance = sorted(Performace.items(), key=operator.itemgetter(1))

In [11]:
sorted_Performance[-10:-1]

[('100.0,1.02,0.72', 0.9610538373424972),
 ('4.0,1.02,0.72', 0.9610538373424972),
 ('0.4,1.02,0.72', 0.9610538373424972),
 ('0.1,1.02,0.72', 0.9610538373424972),
 ('70.0,1.02,0.72', 0.9610538373424972),
 ('0.07,1.02,0.72', 0.9610538373424972),
 ('0.04,1.02,0.72', 0.9610538373424972),
 ('0.7,1.02,0.72', 0.9610538373424972),
 ('1.0,1.02,0.72', 0.9610538373424972)]